In [78]:
import bs4
import requests
import pandas as pd
import html5lib
import numpy as np

In [123]:
wins = [46.5,37.5,25.5,44.5,20.5,57.5,35.5] 
ou = ["over","under","under","over","under","over","under"]

data_dict = {"ou_wins": wins, "over_under": ou, "current_wins": np.nan, "current_losses": np.nan, "projected_wins": np.nan, "looking_good?": ""}

table = pd.DataFrame(
    data_dict,
    index = ['Utah Jazz', 'Milwaukee Bucks', 'Los Angeles Lakers', 'Atlanta Hawks', 'Brooklyn Nets', 'San Antonio Spurs','Miami Heat'],
    columns = ['ou_wins','over_under', 'current_wins', 'current_losses', 'projected_wins', 'looking_good?']
    )

table

,ou_wins,over_under,current_wins,current_losses,projected_wins,looking_good?
Utah Jazz,46.5,over,NaN,NaN,NaN,
Milwaukee Bucks,37.5,under,NaN,NaN,NaN,
Los Angeles Lakers,25.5,under,NaN,NaN,NaN,
Atlanta Hawks,44.5,over,NaN,NaN,NaN,
Brooklyn Nets,20.5,under,NaN,NaN,NaN,
San Antonio Spurs,57.5,over,NaN,NaN,NaN,
Miami Heat,35.5,under,NaN,NaN,NaN,


In [92]:
# Get latest NBA standings
url = 'http://www.espn.com.au/nba/standings'

r = requests.get(url)

souped = bs4.BeautifulSoup(r.text)

st_tb = souped.find_all('table')

#standings_table = pd.read_html(souped)


In [74]:
table_string = str(st_tb)

tbl_list = pd.read_html(table_string)

# Rbind East & West Standings Tables
standings_table = pd.concat([tbl_list[0],tbl_list[1]], ignore_index = True).rename(columns = {"Unnamed: 0":"Team"})


In [124]:
for irow in table.iterrows():
    team = irow[0]
    match_series = standings_table['Team'].str.contains(team)
    idx = match_series[match_series==True].index
    team_row = standings_table.ix[idx][['W','L']]
    
    projected_wins = team_row['W'] * 82 / (team_row['W'] + team_row['L'])
    table.set_value(team, 'current_wins', team_row['W'])
    table.set_value(team, 'current_losses', team_row['L'])
    table.set_value(team, 'projected_wins', projected_wins)
    
    multiplier = np.where(irow[1][1]=='over', 1, -1)
    
    if all((projected_wins - irow[1][0]) * multiplier > 3):
        status = "Good"
    elif all((projected_wins - irow[1][0]) * multiplier > -3):
        status = "Medium"
    else:
        status = "Bad"
            
    table.set_value(team, 'looking_good?', status)
    

table
    

,ou_wins,over_under,current_wins,current_losses,projected_wins,looking_good?
Utah Jazz,46.5,over,1.0,1.0,41.0,Bad
Milwaukee Bucks,37.5,under,2.0,0.0,82.0,Bad
Los Angeles Lakers,25.5,under,1.0,1.0,41.0,Bad
Atlanta Hawks,44.5,over,1.0,1.0,41.0,Bad
Brooklyn Nets,20.5,under,1.0,1.0,41.0,Bad
San Antonio Spurs,57.5,over,1.0,1.0,41.0,Bad
Miami Heat,35.5,under,1.0,1.0,41.0,Bad


# Done!

Just need to convert to py script and add excel export.